In [ ]:
# TODO

# 1. Commented out STP because need to find source of PPP and SHP
# 2. Add graphs

In [47]:
import pandas as pd
from gazpacho import Soup, get

import playerposition as pp

In [49]:
# CBS Data
positionInput = input(f"Input one of: F, C, W, D, G")

position = positionInput.upper()

baseUrl = f"https://www.cbssports.com/fantasy/hockey/stats"
url = f"{baseUrl}/{position}/2021/restofseason/projections"

print(f"URL: {url}")

html = get(url)

Input one of: F, C, W, D, G F


URL: https://www.cbssports.com/fantasy/hockey/stats/F/2021/restofseason/projections


In [50]:
# League Scoring Settings

pointValues = {
    "G": 1.0,
    "A": 0.5,
    "PTS": 0.5,
    "SOG": 0.2,
    # "STP": 0.5,
    "DEF": 0.5,
    "W": 3.0,
    "SV": 0.2,
    "GA": -0.5,
    # "OTL": 1.0,
}

In [51]:
# Invoke this cell if you'd like to change the scoring values of your category


def isFloat(checkInput: str):
    if "." in checkInput:
        split_number = checkInput.split(".")
        if (
            len(split_number) == 2
            and split_number[0].isdigit()
            and split_number[1].isdigit()
        ):
            return True
    return False


# category is one of: G, A, PTS, SOG, STP, DEF, W, SV, GA, OTL
def changeScoringValuesHelper(category: str):
    categoryKey = category.upper()
    if categoryKey not in pointValues.keys():
        return
    categoryVal = input(f"Value of {categoryKey}")
    if isFloat(categoryVal):
        try:
            pointValues[categoryKey] = (
                float(categoryVal) if categoryKey != "GA" else (float(categoryVal) * -1)
            )
            print(f"Successfully updated value of {categoryKey}")
        except:
            print(f"Value not updated")
            pass


def changeScoringValues():
    print(f"Please enter all values as floats (e.g. 1.0)")
    print(f"To skip a category, press ENTER")
    changeScoringValuesHelper("G")
    changeScoringValuesHelper("A")
    changeScoringValuesHelper("PTS")
    changeScoringValuesHelper("SOG")
    # changeScoringValuesHelper("STP")
    changeScoringValuesHelper("DEF")
    changeScoringValuesHelper("W")
    changeScoringValuesHelper("SV")
    changeScoringValuesHelper("GA")
    # changeScoringValuesHelper("OTL")


changeScoringValues()
print(pointValues)

Please enter all values as floats (e.g. 1.0)
To skip a category, press ENTER


Value of G 
Value of A 
Value of PTS 
Value of SOG 
Value of DEF 
Value of W 
Value of SV 
Value of GA 


{'G': 1.0, 'A': 0.5, 'PTS': 0.5, 'SOG': 0.2, 'DEF': 0.5, 'W': 3.0, 'SV': 0.2, 'GA': -0.5}


In [52]:
# Computer individual player's FPTS according to category values

# pointValues = {
#     "G": 1.0,
#     "A": 0.5,
#     "PTS": 0.5,
#     "SOG": 0.2,
#     "STP": 0.5,
#     "DEF": 0.5,
#     "W": 3.0,
#     "SV": 0.2,
#     "GA": -0.5,
#     "OTL": 1.0,
# }


def computeTotalFPTForward(G: int, A: int, PTS: int, SOG: int):
    GVal = G * pointValues["G"]
    AVal = A * pointValues["A"]
    PTSVal = PTS * pointValues["PTS"]
    SOGVal = SOG * pointValues["SOG"]
    result = GVal + AVal + PTSVal + SOGVal
    return result


def computeTotalFPTDefence(G: int, A: int, PTS: int, SOG: int):
    GVal = G * pointValues["G"]
    AVal = A * pointValues["A"]
    PTSVal = PTS * pointValues["PTS"]
    SOGVal = SOG * pointValues["SOG"]
    DEFVal = PTS * pointValues["DEF"]
    result = GVal + AVal + PTSVal + SOGVal + DEFVal
    return result


def computeTotalFPTGoalie(W: int, GA: int, SV: int):
    WVal = W * pointValues["W"]
    GAVal = GA * pointValues["GA"]
    SVVal = SV * pointValues["SV"]
    result = WVal + GAVal + SVVal
    # result = SVVal
    return result

In [60]:
# Hockey-Reference Advanced Stats

ASData = pd.read_csv(f"./data/NHLAdvancedStats_Players2122.csv")

# Parsing Advanced Stats Data
advancedDF = pd.DataFrame(ASData)
cols = [3, 19, 20, 21, 22, 23]
advancedDF = advancedDF[advancedDF.columns[cols]]

print(advancedDF)

                        PLAYER   oZS%  dZS%       TOI/60      TOI(EV)  \
0           Nicholas Abruzzese   46.3  53.7        10:13         9:11   
1                 Noel Acciari     52    48        11:59        10:38   
2                Calen Addison   50.5  49.5        13:47        12:16   
3              Andrew Agozzino     50    50         6:52         6:52   
4                   Jack Ahcan   59.2  40.8        16:02        15:29   
5                Sebastian Aho   57.9  42.1        18:57        14:03   
6                Sebastian Aho   58.8  41.2        16:27        15:40   
7           Alexander Alexeyev     60    40        10:35        10:07   
8                 Wade Allison   66.7  33.3         8:16         7:05   
9               Michael Amadio   51.2  48.8        12:10        11:55   
10               Joey Anderson     75    25         8:46         8:43   
11               Josh Anderson   47.4  52.6        17:14        15:03   
12            Michael Anderson   50.3  49.7        

In [54]:
# Parsing of HTML and extracting 'TableBase-bodyTr' tag
soup = Soup(html)
rows = soup.find("tr", {"class": "TableBase-bodyTr"})
headers = soup.find("tr", {"class": "TableBase-headTr"})

In [ ]:
# Data to Table (Players)
# Default sorting is CBS FPTS


def parse_header(header, pos: str):
    player = header.find("a")[0].text.upper().strip()
    position = "POS"
    team = "TEAM"
    fpts = "FPTS"

    # Skaters
    if pos != "G":
        goalsScored = header.find("a")[4].text.upper()
        assists = header.find("a")[5].text.upper()
        points = header.find("a")[6].text.upper()
        shotsOnGoal = header.find("a")[15].text.upper()
        return player, team, position, fpts, goalsScored, assists, points, shotsOnGoal
    # Goalies
    else:
        wins = header.find("a")[5].text.upper()
        shutouts = header.find("a")[7].text.upper()
        goalsAgainst = header.find("a")[8].text.upper()
        shotsAgainst = header.find("a")[10].text.upper()
        saves = header.find("a")[11].text.upper()
        return (
            player,
            team,
            position,
            fpts,
            wins,
            shutouts,
            goalsAgainst,
            shotsAgainst,
            saves,
        )


def parse_row(row, pos: str):
    # currPlayer is its own class "CellPlayerName--long" in 'TableBase-bodyTr'
    # Find name, position, team for currPlayer
    currPlayer = row.find("span", {"class": "CellPlayerName--long"})
    name = currPlayer.find("a").text
    position = currPlayer.find("span", {"class": "CellPlayerName-position"}).text
    team = currPlayer.find("span", {"class": "CellPlayerName-team"}).text

    # Skaters
    if pos != "G":
        goalsScored = (
            int(row.find("td", {"class": "TableBase-bodyTd"})[4].text)
            if row.find("td", {"class": "TableBase-bodyTd"})[4].text.isnumeric()
            else 0
        )
        assists = (
            int(row.find("td", {"class": "TableBase-bodyTd"})[5].text)
            if row.find("td", {"class": "TableBase-bodyTd"})[5].text.isnumeric()
            else 0
        )
        points = (
            int(row.find("td", {"class": "TableBase-bodyTd"})[6].text)
            if row.find("td", {"class": "TableBase-bodyTd"})[6].text.isnumeric()
            else 0
        )
        shotsOnGoal = (
            int(row.find("td", {"class": "TableBase-bodyTd"})[15].text)
            if row.find("td", {"class": "TableBase-bodyTd"})[15].text.isnumeric()
            else 0
        )

        fpts = (
            computeTotalFPTForward(goalsScored, assists, points, shotsOnGoal)
            if position != "D"
            else computeTotalFPTDefence(goalsScored, assists, points, shotsOnGoal)
        )
        return name, team, position, fpts, goalsScored, assists, points, shotsOnGoal
    # Goalies
    else:
        wins = (
            int(row.find("td", {"class": "TableBase-bodyTd"})[5].text.strip())
            if row.find("td", {"class": "TableBase-bodyTd"})[5].text.strip().isnumeric()
            else 0
        )
        shutouts = (
            int(row.find("td", {"class": "TableBase-bodyTd"})[7].text.strip())
            if row.find("td", {"class": "TableBase-bodyTd"})[7].text.strip().isnumeric()
            else 0
        )
        goalsAgainst = (
            int(row.find("td", {"class": "TableBase-bodyTd"})[8].text)
            if row.find("td", {"class": "TableBase-bodyTd"})[8].text.isnumeric()
            else 0
        )
        saves = (
            int(row.find("td", {"class": "TableBase-bodyTd"})[10].text)
            if row.find("td", {"class": "TableBase-bodyTd"})[10].text.isnumeric()
            else 0
        )
        shotsAgainst = (
            int(row.find("td", {"class": "TableBase-bodyTd"})[11].text)
            if row.find("td", {"class": "TableBase-bodyTd"})[11].text.isnumeric()
            else 0
        )
        fpts = computeTotalFPTGoalie(wins, goalsAgainst, saves)

        return (
            name,
            team,
            position,
            fpts,
            wins,
            shutouts,
            goalsAgainst,
            shotsAgainst,
            saves,
        )


playerInfoDF = pd.DataFrame(
    [parse_row(row, position) for row in rows], columns=parse_header(headers, position)
)
# print(playerInfoDF.columns)
# print(advancedDF.columns)
# print(playerInfoDF.columns.intersection(advancedDF.columns))


if position != "G":
    result = pd.merge(playerInfoDF, advancedDF, on="PLAYER").to_string()
else:
    result = playerInfoDF.to_string()

# Styling options
pd.set_option("display.max_columns", None)  # or 1000
pd.set_option("display.max_rows", None)  # or 1000
pd.set_option("display.max_colwidth", 199)  # or 199


print(result)

In [56]:
# Print results to new file

with open("result.txt", "w") as f:
    print(result, file=f)